In [1]:
import dask.dataframe as dd
# this should whatever your locomizer data file is
#%time temp = dd.read_csv("../data/sample/signals_set_2.csv")
import pandas as pd

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pyarrow.feather as ft

In [6]:
dtype={'reg_share': 'float64'}

In [14]:
%time app_epo = dd.read_csv('PATENTS/Patents/202001_EPO_App_reg.txt', delimiter='|', dtype=dtype)

CPU times: user 37.5 ms, sys: 17.2 ms, total: 54.6 ms
Wall time: 158 ms


In [10]:
%time app_epo.head()

CPU times: user 5.06 s, sys: 1.97 s, total: 7.03 s
Wall time: 13 s


,app_nbr,appln_id,pub_nbr,person_id,app_name,address,reg_code,ctry_code,reg_share,app_share
0,EP20000103094,1,1037159,1,Nokia Corporation,"Keilalahdentie 4,02150 Espoo",FI1B1,FI,1.0,1.0
1,EP20000107845,2,1024191,7,Medical Research Council,"20 Park Crescent,London W1B 1AL",UKI11,GB,1.0,0.5
2,EP20000107845,2,1024191,8,MEDIMMUNE LIMITED,"Milstein Building Granta Park Cambridge,Cambri...",UKH12,GB,1.0,0.5
3,EP20000202556,3,1070969,20,Philips Intellectual Property & Standards GmbH,"Steindamm 94,20099 Hamburg",DE600,DE,1.0,0.5
4,EP20000202556,3,1070969,21,Koninklijke Philips Electronics N.V.,"Groenewoudseweg 1,5621 BA Eindhoven",NL414,NL,1.0,0.5


In [27]:
inv = pd.read_csv('data/inv_pharma_wIPC.csv')

In [16]:
filtered_app_epo = app_epo[app_epo['app_nbr'].isin(inv['app_nbr'])]

In [17]:
filtered_app_epo = filtered_app_epo.compute()

In [18]:
filtered_app_epo.to_csv('app_epo.csv', index=False)

In [13]:
filtered_app_epo=pd.read_csv('app_epo.csv')

In [19]:
filtered_app_epo.head()

,app_nbr,appln_id,pub_nbr,person_id,app_name,address,reg_code,ctry_code,reg_share,app_share
1,EP20000107845,2,1024191,7,Medical Research Council,"20 Park Crescent,London W1B 1AL",UKI11,GB,1.0,0.5
2,EP20000107845,2,1024191,8,MEDIMMUNE LIMITED,"Milstein Building Granta Park Cambridge,Cambri...",UKH12,GB,1.0,0.5
571,EP20080159140,589,2058311,1383,Boehringer Ingelheim Pharma GmbH & Co. KG,"Binger Strasse 173,55216 Ingelheim am Rhein",DEB3J,DE,1.0,1.0
572,EP20080159141,590,2070539,1383,Boehringer Ingelheim Pharma GmbH & Co. KG,"Binger Strasse 173,55216 Ingelheim am Rhein",DEB3J,DE,1.0,1.0
1046,EP20010940394,1254,1280510,2800,Unilever PLC,"Unilever House 100 Victoria Embankment London,...",UKI11,GB,1.0,0.5


In [22]:
len(filtered_app_epo['app_nbr'].unique())

31283

In [25]:
len(inv['app_nbr'].unique())

31445

In [26]:
inv_not_in = inv[~inv['app_nbr'].isin(filtered_app_epo['app_nbr'])]

In [27]:
len(inv_not_in['app_nbr'].unique())

162

In [29]:
inv_not_in.to_csv('app_not_in_epo.csv', index=False)

In [3]:
inv_not_in = pd.read_csv('app_not_in_epo.csv')

In [4]:
inv_not_in.head()

,Unnamed: 0,app_nbr,appln_id,person_id,inv_name,address,reg_code,ctry_code,prio_year,app_year,IPC
0,92640,EP20000932586,15769302.0,917423.0,"ASHCROFT, Gillian, S.","65 Orrell Lane, Burscough,Nr. Ormskirk, Lancas...",UKD43,GB,2000.0,2000.0,A61K038/00
1,92641,EP20000932586,15769302.0,917423.0,"ASHCROFT, Gillian, S.","65 Orrell Lane, Burscough,Nr. Ormskirk, Lancas...",UKD43,GB,2000.0,2000.0,A61K038/17
2,92642,EP20000932586,15769302.0,917423.0,"ASHCROFT, Gillian, S.","65 Orrell Lane, Burscough,Nr. Ormskirk, Lancas...",UKD43,GB,2000.0,2000.0,A61K045/00
3,92643,EP20000932586,15769302.0,917423.0,"ASHCROFT, Gillian, S.","65 Orrell Lane, Burscough,Nr. Ormskirk, Lancas...",UKD43,GB,2000.0,2000.0,A61P017/02
4,92644,EP20000932586,15769302.0,917423.0,"ASHCROFT, Gillian, S.","65 Orrell Lane, Burscough,Nr. Ormskirk, Lancas...",UKD43,GB,2000.0,2000.0,A61P019/04


In [5]:
inv_not_in['ctry_code'].unique()

array(['GB', nan], dtype=object)

In [31]:
%time epo_pct = dd.read_csv('PATENTS/Patents/202001_EPO_PCT.txt', delimiter='|')

CPU times: user 22.4 ms, sys: 2.94 ms, total: 25.4 ms
Wall time: 22.8 ms


In [32]:
filtered_epo_pct = epo_pct[epo_pct['app_nbr'].isin(inv_not_in['app_nbr'])]

In [33]:
filtered_epo_pct = filtered_epo_pct.compute()

In [34]:
len(filtered_epo_pct['app_nbr'].unique())

162

In [35]:
filtered_epo_pct.to_csv('epo_pct_app.csv', index=False)

In [15]:
filtered_epo_pct=pd.read_csv('epo_pct_app.csv')

In [7]:
dtype={'app_share': 'float64',
       'appln_id': 'float64',
      'reg_share': 'float64'}

In [8]:
%time app_pct = dd.read_csv('PATENTS/Patents/202001_PCT_App_reg.txt', delimiter='|', dtype=dtype)

CPU times: user 45.5 ms, sys: 9.03 ms, total: 54.5 ms
Wall time: 168 ms


In [9]:
filtered_app_pct = app_pct[app_pct['pct_nbr'].isin(filtered_epo_pct['pct_nbr'])]
filtered_app_pct = filtered_app_pct[filtered_app_pct['ctry_code'] == 'GB']

In [10]:
filtered_app_pct.head()

,pct_nbr,internat_appln_nr,appln_id,app_name,address,reg_code,ctry_code,reg_share,app_share
104229,WO1992004469,WO1991GB01517,23032911.0,IMPERIAL CHEMICAL INDUSTRIES PLC,London SW1P 3JF,UKI11,GB,1.0,1.0
127066,WO1993003042,WO1992GB01306,23062471.0,SMITHKLINE BEECHAM PLC,Middlesex TW8 9EP,UKI23,GB,1.0,1.0
194701,WO1995009915,WO1994GB02172,23123494.0,NBL GENE SCIENCES LIMITED,Northumberland NE23 9HL,UKC21,GB,1.0,1.0
223379,WO1996000785,WO1995GB01517,23151498.0,The Scottish Agricultural College,Ayr KA6 5HW,UKM37,GB,1.0,0.5
223380,WO1996000785,WO1995GB01517,23151498.0,THE VICTORIA UNIVERSITY OF MANCHESTER,Manchester M13 9PL,UKD31,GB,1.0,0.5


In [7]:
len(filtered_app_pct)

179

In [11]:
filtered_app_pct = filtered_app_pct.compute()

In [12]:
filtered_app_pct.to_csv('app_pct.csv', index=False)

In [14]:
len(filtered_app_pct['pct_nbr'].unique())

105

In [16]:
filtered_app_pct = pd.merge(filtered_app_pct, filtered_epo_pct, on='pct_nbr', how='left')

In [17]:
filtered_app_pct.head()

,pct_nbr,internat_appln_nr,appln_id,app_name,address,reg_code,ctry_code,reg_share,app_share,app_nbr
0,WO1992004469,WO1991GB01517,23032911.0,IMPERIAL CHEMICAL INDUSTRIES PLC,London SW1P 3JF,UKI11,GB,1.0,1.0,EP19910916485
1,WO1993003042,WO1992GB01306,23062471.0,SMITHKLINE BEECHAM PLC,Middlesex TW8 9EP,UKI23,GB,1.0,1.0,EP19920915476
2,WO1995009915,WO1994GB02172,23123494.0,NBL GENE SCIENCES LIMITED,Northumberland NE23 9HL,UKC21,GB,1.0,1.0,EP19940928473
3,WO1996000785,WO1995GB01517,23151498.0,The Scottish Agricultural College,Ayr KA6 5HW,UKM37,GB,1.0,0.5,EP19950923439
4,WO1996000785,WO1995GB01517,23151498.0,THE VICTORIA UNIVERSITY OF MANCHESTER,Manchester M13 9PL,UKD31,GB,1.0,0.5,EP19950923439


In [18]:
filtered_app_epo.head()

,app_nbr,appln_id,pub_nbr,person_id,app_name,address,reg_code,ctry_code,reg_share,app_share
0,EP20000107845,2,1024191,7,Medical Research Council,"20 Park Crescent,London W1B 1AL",UKI11,GB,1.0,0.5
1,EP20000107845,2,1024191,8,MEDIMMUNE LIMITED,"Milstein Building Granta Park Cambridge,Cambri...",UKH12,GB,1.0,0.5
2,EP20080159140,589,2058311,1383,Boehringer Ingelheim Pharma GmbH & Co. KG,"Binger Strasse 173,55216 Ingelheim am Rhein",DEB3J,DE,1.0,1.0
3,EP20080159141,590,2070539,1383,Boehringer Ingelheim Pharma GmbH & Co. KG,"Binger Strasse 173,55216 Ingelheim am Rhein",DEB3J,DE,1.0,1.0
4,EP20010940394,1254,1280510,2800,Unilever PLC,"Unilever House 100 Victoria Embankment London,...",UKI11,GB,1.0,0.5


In [21]:
filtered_app_epo = filtered_app_epo[['app_nbr', 'appln_id', 'app_name','address','reg_code','ctry_code','reg_share','app_share']]

In [22]:
filtered_app_pct = filtered_app_pct[['app_nbr', 'appln_id', 'app_name','address','reg_code','ctry_code','reg_share','app_share']]

In [24]:
filtered_app = pd.concat([filtered_app_epo, filtered_app_pct])

In [25]:
filtered_app.head()

,app_nbr,appln_id,app_name,address,reg_code,ctry_code,reg_share,app_share
0,EP20000107845,2.0,Medical Research Council,"20 Park Crescent,London W1B 1AL",UKI11,GB,1.0,0.5
1,EP20000107845,2.0,MEDIMMUNE LIMITED,"Milstein Building Granta Park Cambridge,Cambri...",UKH12,GB,1.0,0.5
2,EP20080159140,589.0,Boehringer Ingelheim Pharma GmbH & Co. KG,"Binger Strasse 173,55216 Ingelheim am Rhein",DEB3J,DE,1.0,1.0
3,EP20080159141,590.0,Boehringer Ingelheim Pharma GmbH & Co. KG,"Binger Strasse 173,55216 Ingelheim am Rhein",DEB3J,DE,1.0,1.0
4,EP20010940394,1254.0,Unilever PLC,"Unilever House 100 Victoria Embankment London,...",UKI11,GB,1.0,0.5


In [28]:
filtered_app.to_csv('data/app_reg_pharma.csv', index = False)

In [29]:
inv_not_in = inv[~inv['app_nbr'].isin(filtered_app['app_nbr'])]

In [32]:
len(inv_not_in['app_nbr'].unique())

57